## EDA

Working with the data...

1. Generate a word cloud of the abstracts and filling out the common words

2. Generate a basic hist of the labels identified and classified by LitCovid.

3. 

In [3]:
# packages to load in
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import json


import matplotlib.pyplot as plt

In [18]:
# load in my shit
f = open('data/train.json')
data = json.load(f)
data = dict(data)

In [21]:
data['PMID:32323279']

{'input': '[Renin-Angiotensin-System (RAS) and COVID-19 - On The Prescription of RAS Blockers].Twenty years ago, an enzyme homologous to the previously known angiotensin-converting enzyme (ACE) was identified, and subsequently named ACE2. In the renin-angiotensin system (RAS), ACE2 has counter-regulatory functions against the classical effector peptide angiotensin II, for example in blood pressure regulation and cardiovascular remodeling. However, ACE2 provides an initially unexpected interesting link between virology and cardiovascular medicine. That is, ACE2 represents the binding receptor for the cellular uptake of SARS-CoV and SARS-CoV-2 viruses. Thus, ACE2 is relevant for COVID-19. In this context, it was suspected that therapy with RAS blockers might promote transmission and complications of COVID-19 by upregulation of ACE2 expression. The aim of this short review is, to describe the link between the RAS, particularly ACE2, and COVID-19. Based on our analysis and evaluation of th